First download/import LogicCircuits

In [1]:
# import Pkg; Pkg.add("LogicCircuits") # uncomment this if you have'n LogicCircuits
using LogicCircuits 

Now let us "use" our local package (do not add/install this or you'll be using "their" package)

In [2]:
using ProbabilisticCircuits

Now let's load some example psdd from the repository

In [4]:
psdd = zoo_psdd("little_4var.psdd"); # PSDD over 4 variables

A PSDD is internally simply a vector of ProbNodes

In [47]:
psdd isa Vector{<:ProbΔNode}

true

We can print some information about the PSDD

In [18]:
println("PSDD has $(size(psdd)[1]) nodes.")
println("PSDD has $(num_parameters(psdd)) parameters.")

PSDD has 20 nodes.
PSDD has 9 parameters.


Now let's computes the marginal probability of some evidence.
We first create a one-row Matrix `XData` containing assigments for each variable. 1 means `True`, 0 means `False`, -1 means missing or marginalized.

In [11]:
evidence = XData(Int8.([1 1 -1 -1]));

Now compute the log probability of that evidence.

In [13]:
logprob = log_proba(psdd, evidence) # note that the output is an array since the input is also an array

1-element Array{Float64,1}:
 -2.3025850929940455

To convert it into a probability value we just need to exponentiate it.  

In [14]:
exp(logprob[1])

0.10000000000000002

We can compute probabilities for multiple evidences

In [23]:
data = XData(Int8.([1 1 -1 -1; 
                    1 0 -1 -1; 
                    0 1 -1 -1; 
                    0 0 -1 -1]))
exp.(log_proba(psdd,data)) # the values should add up to one.

4-element Array{Float64,1}:
 0.10000000000000002
 0.10000000000000002
 0.10000000000000002
 0.6999999999999998 

This generates all possible complete evidences, then compute their probabilities

In [35]:
sample_space = XData(generate_data_all(4)) # because we have 4 variables
probs = exp.(log_proba(psdd,sample_space))
sum(probs) # ≈ 1

1.0000000000000002

Let's generate some data using the PSDD

In [42]:
using DataStructures # to use a dictionary for the tally
hist = DefaultDict{AbstractString,Float64}(0.0) # instance tally 

N = 1000 # no. of samples
for i = 1:N
    cur = join(Int.(sample(psdd))) # string representation of evidence
    hist[cur] += 1
end

In [43]:
for k in keys(hist)
        hist[k] /= N       
end

In [44]:
 for k in keys(hist)
        cur = parse(Int32, k, base=2) + 1
        println("Prob($k) = $(probs[cur]) ≈ $(hist[k])")
 end

Prob(0011) = 0.14 ≈ 0.153
Prob(0110) = 0.029999999999999995 ≈ 0.036
Prob(1100) = 0.010000000000000005 ≈ 0.002
Prob(1011) = 0.02000000000000001 ≈ 0.023
Prob(0001) = 0.27999999999999997 ≈ 0.293
Prob(1001) = 0.04000000000000001 ≈ 0.041
Prob(1110) = 0.029999999999999995 ≈ 0.019
Prob(1101) = 0.04000000000000001 ≈ 0.037
Prob(1010) = 0.029999999999999995 ≈ 0.022
Prob(0010) = 0.20999999999999996 ≈ 0.212
Prob(0101) = 0.04000000000000001 ≈ 0.039
Prob(1111) = 0.02000000000000001 ≈ 0.019
Prob(1000) = 0.010000000000000005 ≈ 0.005
Prob(0100) = 0.010000000000000005 ≈ 0.008
Prob(0000) = 0.07 ≈ 0.07
Prob(0111) = 0.02000000000000001 ≈ 0.021


We can print out the PSDD as a dot file (then generate a picture using graphviz's dot command)

In [ ]:
save_as_dot(psdd, "psdd.dot")

Assuming you have run "dot -Tpng -O psdd.dot" in the examples folder, we can draw the corresponding image.

<img src="psdd.dot.png" />